# このファイルは、最初のデータから解析用のデータに変換できるコードとなっています。
# 最新コードに更新済み。
# ・アーカイブ入り・

In [1]:
# !pip install japanize-matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
import scipy.stats as stats
import scikit_posthocs as sp
import numpy as np
import io
import japanize_matplotlib
from statsmodels.stats.contingency_tables import Table

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [10]:
data0 = pd.read_csv("/Users/ayo/Desktop/_GSAIS_/Research/OralHealth_tokyo/paper_analysis/data/analysisData_20251128.csv")
# Convert 'date' column to datetime objects
data0['date'] = pd.to_datetime(data0['date'])
data0.head()

,No_All,CGC,date,sex,age_year,age_month,age,abuse_1,abuse,abuse_num,...,wake_up,breakfast,morning_brushing,school,bedtime,night_brushing,TV,game,meal,extra_lesson
0,1,tokyo,2016-10-12,Female,16,3,16.250000,NaN,6.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,tokyo,2016-10-12,Female,14,6,14.500000,NaN,5.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,tokyo,2016-10-12,Female,15,1,15.083333,NaN,1.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,tokyo,2016-10-12,Female,14,5,14.416667,NaN,3.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,tokyo,2016-10-12,Female,15,10,15.833333,NaN,2.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Get a summary of statistics (count, mean, std, min, 25%, 50%, 75%, max)
data0.describe()

,No_All,date,age_year,age_month,age,abuse_1,abuse,abuse_num,U17,U16,...,wake_up,breakfast,morning_brushing,school,bedtime,night_brushing,TV,game,meal,extra_lesson
count,2480.000000,2480,2480.000000,2480.000000,2480.000000,99.000000,2474.000000,2480.000000,1122.000000,2023.000000,...,429.000000,429.000000,429.000000,428.000000,429.000000,428.000000,429.000000,429.000000,429.000000,429.000000
mean,1240.500000,2021-09-14 03:04:38.709677312,10.775403,5.347581,11.221035,3.353535,3.204527,1.037500,0.222816,0.243697,...,2.351981,1.573427,1.738928,1.502336,3.111888,1.257009,1.524476,1.489510,1.655012,1.550117
min,1.000000,2016-10-12 00:00:00,1.000000,0.000000,2.083333,1.000000,1.000000,0.000000,-1.000000,-1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,620.750000,2019-09-11 00:00:00,8.000000,2.000000,8.333333,2.000000,1.000000,1.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,1240.500000,2022-01-12 00:00:00,12.000000,5.000000,12.083333,3.000000,3.000000,1.000000,0.000000,0.000000,...,2.000000,1.000000,1.000000,1.000000,3.000000,1.000000,2.000000,1.000000,1.000000,2.000000
75%,1860.250000,2023-06-14 00:00:00,14.000000,8.000000,14.250000,5.000000,5.000000,1.000000,0.000000,0.000000,...,2.000000,2.000000,2.000000,2.000000,4.000000,1.000000,2.000000,2.000000,2.000000,2.000000
max,2480.000000,2025-02-20 00:00:00,17.000000,15.000000,17.916667,7.000000,7.000000,2.000000,3.000000,6.000000,...,6.000000,22.000000,33.000000,4.000000,8.000000,3.000000,2.000000,3.000000,3.000000,3.000000
std,716.058657,NaN,3.993434,3.494385,3.990107,1.791779,1.995396,0.202359,0.835377,0.695775,...,1.578070,1.214147,1.714943,0.754375,1.270283,0.538297,0.499984,0.509725,0.768756,0.507358


In [12]:
print(len(data0.columns))
# Get column names
data0.columns

80


Index(['No_All', 'CGC', 'date', 'sex', 'age_year', 'age_month', 'age',
       'abuse_1', 'abuse', 'abuse_num', 'U17', 'U16', 'U15', 'U14', 'U13',
       'U12', 'U11', 'U21', 'U22', 'U23', 'U24', 'U25', 'U26', 'U27', 'L37',
       'L36', 'L35', 'L34', 'L33', 'L32', 'L31', 'L41', 'L42', 'L43', 'L44',
       'L45', 'L46', 'L47', 'u55', 'u54', 'u53', 'u52', 'u51', 'u61', 'u62',
       'u63', 'u64', 'u65', 'l75', 'l74', 'l73', 'l72', 'l71', 'l81', 'l82',
       'l83', 'l84', 'l85', 'needTOBEtreated', 'emergency',
       'emergencyInMonths', 'gingivitis', 'occlusalRelationship', 'habits',
       'OralCleanStatus', 'instruction_detail', 'instruction', 'memo',
       'dentists', '\ndental_hygienist', 'wake_up', 'breakfast',
       'morning_brushing', 'school', 'bedtime', 'night_brushing', 'TV', 'game',
       'meal', 'extra_lesson'],
      dtype='object')

In [13]:
# Define the mappings
abuse_map = {
    1: "Physical Abuse",
    2: "Neglect",
    3: "Emotional Abuse",
    4: "Sexual Abuse",
    5: "Delinquency",
    6: "Parenting Difficulties",
    7: "Others"
}

occlusal_map = {
    1: "Normal Occlusion",
    2: "Crowding",
    3: "Anterior Crossbite",
    4: "Open Bite",
    5: "Maxillary Protrusion",
    6: "Crossbite",
    7: "Others"
}

need_treated_map = {
    1: "No Treatment Required",
    2: "Treatment Required"
}

emergency_map = {
    1: "Urgent Treatment Required"
}

gingivitis_map = {
    1: "No Gingivitis",
    2: "Gingivitis"
}

oral_clean_map = {
    1: "Poor",
    2: "Fair",
    3: "Good"
}

habits_map = {
    1: "None",
    2: "Digit Sucking",
    3: "Nail biting",
    4: "Tongue Thrusting",
    5: "Smoking",
    6: "Others"
}

# Apply the mappings
# Using .replace() ensures that any values not listed (like NaNs) remain unchanged
data0['abuse'] = data0['abuse'].replace(abuse_map)
data0['occlusalRelationship'] = data0['occlusalRelationship'].replace(occlusal_map)
data0['needTOBEtreated'] = data0['needTOBEtreated'].replace(need_treated_map)
data0['emergency'] = data0['emergency'].replace(emergency_map)
data0['gingivitis'] = data0['gingivitis'].replace(gingivitis_map)
data0['OralCleanStatus'] = data0['OralCleanStatus'].replace(oral_clean_map)
data0['habits'] = data0['habits'].replace(habits_map)

# Verify the changes
cols_to_check = ['abuse', 'occlusalRelationship', 'needTOBEtreated', 'emergency', 'gingivitis', 'OralCleanStatus', 'habits']
print(data0[cols_to_check].head())

                    abuse  occlusalRelationship        needTOBEtreated  \
0  Parenting Difficulties              Crowding     Treatment Required   
1             Delinquency  Maxillary Protrusion     Treatment Required   
2          Physical Abuse              Crowding  No Treatment Required   
3         Emotional Abuse      Normal Occlusion     Treatment Required   
4                 Neglect      Normal Occlusion     Treatment Required   

  emergency     gingivitis OralCleanStatus   habits  
0       NaN     Gingivitis            Poor     None  
1       NaN     Gingivitis            Fair  Smoking  
2       NaN     Gingivitis            Fair     None  
3       NaN  No Gingivitis            Good     None  
4       NaN  No Gingivitis            Fair     None  


In [14]:
# 1. Define the logical order for each column
abuse_order = [
    "Physical Abuse", "Neglect", "Emotional Abuse", "Sexual Abuse", 
    "Delinquency", "Parenting Difficulties", "Others"
]
occlusal_order = [
    "Normal Occlusion", "Crowding", "Anterior Crossbite", "Open Bite", 
    "Maxillary Protrusion", "Crossbite", "Others"
]
need_treated_order = ["No Treatment Required", "Treatment Required"]
emergency_order = ["Urgent Treatment Required"]
gingivitis_order = ["No Gingivitis", "Gingivitis"]
oral_clean_order = ["Poor", "Fair", "Good"]
habits_order = [
    "None", "Digit Sucking", "Nail biting", "Tongue Thrusting", 
    "Smoking", "Others"
]

# 2. Convert columns to Categorical with the specified order
# (Replace 'data_noNA_Only1' with your current dataframe variable name if different)
target_df = data0

target_df['abuse'] = pd.Categorical(target_df['abuse'], categories=abuse_order, ordered=True)
target_df['occlusalRelationship'] = pd.Categorical(target_df['occlusalRelationship'], categories=occlusal_order, ordered=True)
target_df['needTOBEtreated'] = pd.Categorical(target_df['needTOBEtreated'], categories=need_treated_order, ordered=True)
target_df['emergency'] = pd.Categorical(target_df['emergency'], categories=emergency_order, ordered=True)
target_df['gingivitis'] = pd.Categorical(target_df['gingivitis'], categories=gingivitis_order, ordered=True)
target_df['OralCleanStatus'] = pd.Categorical(target_df['OralCleanStatus'], categories=oral_clean_order, ordered=True)
target_df['habits'] = pd.Categorical(target_df['habits'], categories=habits_order, ordered=True)

print("Columns converted to ordered categories.")

Columns converted to ordered categories.


In [15]:
output_path = "/Users/ayo/Desktop/_GSAIS_/Research/OralHealth_tokyo/paper_analysis/data/AllData_aftermapping.csv"
data0.to_csv(output_path, index=False)

### All Data (until 2025)

In [17]:
# 1. Create a list to store the results
summary_data = []

# 2. Iterate through each column to count values
for col in data0.columns[1:65]:
    # value_counts(dropna=False) includes NaN/missing values in the count
    counts = data0[col].value_counts(dropna=False).sort_index()
    
    for value, count in counts.items():
        summary_data.append({
            "Column": col,
            "Value": value,
            "Count": count
        })

# 3. Convert to a DataFrame
df_summary = pd.DataFrame(summary_data)

# 4. Save to CSV
output_path = "/Users/ayo/Desktop/_GSAIS_/Research/OralHealth_tokyo/paper_analysis/result/AllData_columns_value_counts.csv"
df_summary.to_csv(output_path, index=False)

print(f"Saved value counts to: {output_path}")
print(df_summary.head())

Saved value counts to: /Users/ayo/Desktop/_GSAIS_/Research/OralHealth_tokyo/paper_analysis/result/AllData_columns_value_counts.csv
  Column                Value  Count
0    CGC             itabashi     79
1    CGC                tokyo   2144
2    CGC            wakamatsu    257
3   date  2016-10-12 00:00:00     33
4   date  2018-03-14 00:00:00     31


### All Data (2016-2024/3)

In [18]:
# Filter data for dates on or before March 31, 2024
#    (This includes all of March 2024)
data_untilMar2024 = data0[data0['date'] <= '2024-03-31']

# Check the results
print(f"Original row count: {len(data0)}")
print(f"Filtered row count: {len(data_untilMar2024)}")
print(f"Date range in filtered data: {data_untilMar2024['date'].min()} to {data_untilMar2024['date'].max()}")

# Save the filtered data to a new CSV file
output_path = "/Users/ayo/Desktop/_GSAIS_/Research/OralHealth_tokyo/paper_analysis/data/AllData_tillMar2024.csv"
data_untilMar2024.to_csv(output_path, index=False)

Original row count: 2480
Filtered row count: 2162
Date range in filtered data: 2016-10-12 00:00:00 to 2024-03-21 00:00:00


In [ ]:
# 1. Create a list to store the results
summary_data = []

# 2. Iterate through each column to count values
for col in data_untilMar2024.columns[1:65]:
    # value_counts(dropna=False) includes NaN/missing values in the count
    counts = data_untilMar2024[col].value_counts(dropna=False).sort_index()
    
    for value, count in counts.items():
        summary_data.append({
            "Column": col,
            "Value": value,
            "Count": count
        })

# 3. Convert to a DataFrame
df_summary = pd.DataFrame(summary_data)

# 4. Save to CSV
output_path = "/Users/ayo/Desktop/_GSAIS_/Research/OralHealth_tokyo/paper_analysis/result/DataUntilMar2024_columns_value_counts.csv"
df_summary.to_csv(output_path, index=False)

print(f"Saved value counts to: {output_path}")
print(df_summary.head())

Saved value counts to: /Users/yining/Desktop/_GSAIS_/Research/OralHealth_tokyo/paper_analysis/result/DataUntilMar2024_columns_value_counts.csv
  Column                Value  Count
0    CGC             itabashi     79
1    CGC                tokyo   1894
2    CGC            wakamatsu    189
3   date  2016-10-12 00:00:00     33
4   date  2018-03-14 00:00:00     31


### No Abuse NA

In [82]:
# Keep only rows where abuse_num is NOT 0
data_untilMar2024_noAbuseNA = data_untilMar2024[data_untilMar2024['abuse_num'] != 0]

# Verify the result
print(f"Remaining rows: {len(data_untilMar2024_noAbuseNA)}")
print(f"Unique values in abuse_num: {data_untilMar2024_noAbuseNA['abuse_num'].unique()}")

Remaining rows: 2156
Unique values in abuse_num: [1 2]


In [83]:
# 1. Create a list to store the results
summary_data = []

# 2. Iterate through each column to count values
for col in data_untilMar2024_noAbuseNA.columns[1:65]:
    # value_counts(dropna=False) includes NaN/missing values in the count
    counts = data_untilMar2024_noAbuseNA[col].value_counts(dropna=False).sort_index()
    
    for value, count in counts.items():
        summary_data.append({
            "Column": col,
            "Value": value,
            "Count": count
        })

# 3. Convert to a DataFrame
df_summary = pd.DataFrame(summary_data)

# 4. Save to CSV
output_path = "/Users/yining/Desktop/_GSAIS_/Research/OralHealth_tokyo/paper_analysis/result/DataUntilMar2024_NoAbuseNA_columns_value_counts.csv"
df_summary.to_csv(output_path, index=False)

print(f"Saved value counts to: {output_path}")
print(df_summary.head())

Saved value counts to: /Users/yining/Desktop/_GSAIS_/Research/OralHealth_tokyo/paper_analysis/result/DataUntilMar2024_NoAbuseNA_columns_value_counts.csv
  Column                Value  Count
0    CGC             itabashi     79
1    CGC                tokyo   1888
2    CGC            wakamatsu    189
3   date  2016-10-12 00:00:00     32
4   date  2018-03-14 00:00:00     31


### Only 1-type Abuse(1-4)

In [84]:
# Filter data based on the two conditions
# 1. abuse_num is 1
# 2. abuse is between 1 and 4 (inclusive)
target_abuse_types = [
    "Physical Abuse", 
    "Neglect", 
    "Emotional Abuse", 
    "Sexual Abuse"
]

data_noNA_Only1 = data_untilMar2024_noAbuseNA[
    (data_untilMar2024_noAbuseNA['abuse_num'] == 1) & 
    (data_untilMar2024_noAbuseNA['abuse'].isin(target_abuse_types))
]

# Verify the results
print(f"Filtered row count: {len(data_noNA_Only1)}")
print(f"Unique abuse_num values: {data_noNA_Only1['abuse_num'].unique()}")
print(f"Unique abuse values: {sorted(data_noNA_Only1['abuse'].unique())}")

Filtered row count: 1235
Unique abuse_num values: [1]
Unique abuse values: ['Emotional Abuse', 'Neglect', 'Physical Abuse', 'Sexual Abuse']


In [85]:
# 1. Create a list to store the results
summary_data = []

# 2. Iterate through each column to count values
for col in data_noNA_Only1.columns[1:65]:
    # value_counts(dropna=False) includes NaN/missing values in the count
    counts = data_noNA_Only1[col].value_counts(dropna=False).sort_index()
    
    for value, count in counts.items():
        summary_data.append({
            "Column": col,
            "Value": value,
            "Count": count
        })

# 3. Convert to a DataFrame
df_summary = pd.DataFrame(summary_data)

# 4. Save to CSV
output_path = "/Users/yining/Desktop/_GSAIS_/Research/OralHealth_tokyo/paper_analysis/result/noNA_Only1_columns_value_counts.csv"
df_summary.to_csv(output_path, index=False)

print(f"Saved value counts to: {output_path}")
print(df_summary.head())

Saved value counts to: /Users/yining/Desktop/_GSAIS_/Research/OralHealth_tokyo/paper_analysis/result/noNA_Only1_columns_value_counts.csv
  Column                Value  Count
0    CGC             itabashi     47
1    CGC                tokyo   1057
2    CGC            wakamatsu    131
3   date  2016-10-12 00:00:00     17
4   date  2018-03-14 00:00:00     16


## Final Data

In [86]:
df = data_noNA_Only1
save_path = '/Users/yining/Desktop/_GSAIS_/Research/OralHealth_tokyo/paper_analysis/data/data_noNA_OnlyAbuse.csv'
df.to_csv(save_path, index=False)

print(f"Row Count: {len(df)}")
print(f"Column Count: {len(df.columns)}")
print(f"Columns Names: {df.columns}")
print(f"abuse_num: {df['abuse_num'].unique()}")
print(f"Unique abuse values: {sorted(df['abuse'].unique())}")
print(df['abuse'].value_counts())

df.head()

Row Count: 1235
Column Count: 80
Columns Names: Index(['No_All', 'CGC', 'date', 'sex', 'age_year', 'age_month', 'age',
       'abuse_1', 'abuse', 'abuse_num', 'U17', 'U16', 'U15', 'U14', 'U13',
       'U12', 'U11', 'U21', 'U22', 'U23', 'U24', 'U25', 'U26', 'U27', 'L37',
       'L36', 'L35', 'L34', 'L33', 'L32', 'L31', 'L41', 'L42', 'L43', 'L44',
       'L45', 'L46', 'L47', 'u55', 'u54', 'u53', 'u52', 'u51', 'u61', 'u62',
       'u63', 'u64', 'u65', 'l75', 'l74', 'l73', 'l72', 'l71', 'l81', 'l82',
       'l83', 'l84', 'l85', 'needTOBEtreated', 'emergency',
       'emergencyInMonths', 'gingivitis', 'occlusalRelationship', 'habits',
       'OralCleanStatus', 'instruction_detail', 'instruction', 'memo',
       'dentists', '\ndental_hygienist', 'wake_up', 'breakfast',
       'morning_brushing', 'school', 'bedtime', 'night_brushing', 'TV', 'game',
       'meal', 'extra_lesson'],
      dtype='object')
abuse_num: [1]
Unique abuse values: ['Emotional Abuse', 'Neglect', 'Physical Abuse', 'Sexual

,No_All,CGC,date,sex,age_year,age_month,age,abuse_1,abuse,abuse_num,...,wake_up,breakfast,morning_brushing,school,bedtime,night_brushing,TV,game,meal,extra_lesson
2,3,tokyo,2016-10-12,Female,15,1,15.083333,NaN,Physical Abuse,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,tokyo,2016-10-12,Female,14,5,14.416667,NaN,Emotional Abuse,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,tokyo,2016-10-12,Female,15,10,15.833333,NaN,Neglect,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,tokyo,2016-10-12,Female,15,0,15.000000,NaN,Physical Abuse,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,tokyo,2016-10-12,Female,17,0,17.000000,NaN,Physical Abuse,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
